In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1621917982298_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [2]:
textcorporalocation = 's3://cpsc5330s21/data-input/textcorpora-SMALL/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import re
def filename(path):
  return re.match('s3://cpsc5330s21/data-input/textcorpora-SMALL/(.+)\.txt', path)[1]

def cleanUpString(token):
    return "".join(filter(lambda c: 97 <= ord(c) <= 122 or c == " ", token.lower()))
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql.functions import explode, col
def indexDocuments(file_location):
    f = sc.wholeTextFiles(file_location)
    doc_id_and_all_its_terms = spark.createDataFrame(f.map(lambda t: (filename(t[0]), cleanUpString(t[1]).split())), ["docid", "file_contents"])
    #  doc_id_with_split_text.take(1) would print: [('austen-emma', ['[Emma', 'by', 
    #           'Jane', 'Austen', '1816]', 'VOLUME', 'I', 'CHAPTER', 
    #           'I', 'Emma', 'Woodhouse,', 'handsome,', 'clever,', 'and',
    #           'rich,', 'with', 'a', 'comfortable', 'home', 'and', 'happy',...         
    # doc_id_with_split_text.take(1)
    
    docid_term_pair = doc_id_and_all_its_terms.select(doc_id_and_all_its_terms.docid, explode(doc_id_and_all_its_terms.file_contents)).withColumnRenamed("col", "term")
    #docid_term_pair.show()
    # +-----------+-----------+
    # |      docid|       term|
    # +-----------+-----------+
    # |austen-emma|       emma|
    # |austen-emma|         by|
    # |austen-emma|       jane|
    # |austen-emma|     austen|
    
    #from pyspark.sql.functions import groupBy
    doc_id_term_token_counts = docid_term_pair.groupBy("docid", "term").count().withColumnRenamed("count", "docid_term_count")
    #doc_id_term_token_counts.show()
    # +-----------+--------------+----------------+
    # |      docid|          term|docid_term_count|
    # +-----------+--------------+----------------+
    # |austen-emma|            on|             572|
    # |austen-emma|          busy|              18|
    # |austen-emma|         feels|              10|
    # |austen-emma|         askme|               1|
    
    docid_term_count = docid_term_pair.groupBy("docid").count().withColumnRenamed("count", "all_token_count")
    #docid_term_count.orderBy(col("all_token_count").desc()).show()
    # +-------------------+---------------+
    # |              docid|all_token_count|
    # +-------------------+---------------+
    # |          bible-kjv|         740800|
    # | melville-moby_dick|         192326|
    # |        austen-emma|         143896|
    # |     whitman-leaves|         111016|
    
    term_idf = doc_id_term_token_counts.groupBy("term").count().withColumnRenamed("count", "term_in_num_docs_count")
    #term_idf.sort(col("term")).show()
    # +------------------+----------------------+
    # |              term|term_in_num_docs_count|
    # +------------------+----------------------+
    # |                 a|                    10|
    # |             aaron|                     2|
    # |          aaronand|                     1|
    # |         aaronites|                     1|
    # |         aaronmade|                     1|
    # |        aaronnadab|                     1|
    
    pre_tfidf_calc = doc_id_term_token_counts.\
        join(term_idf, ["term"]).\
        join(docid_term_count, ["docid"])
    
    tfidf = pre_tfidf_calc.\
        withColumn("tfidf", 1000000*(pre_tfidf_calc.docid_term_count/pre_tfidf_calc.all_token_count)/pre_tfidf_calc.term_in_num_docs_count).\
        select(["term", "docid", "tfidf"])
    
    return tfidf


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
myindex = indexDocuments(textcorporalocation)
myindex.persist()
myindex.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+------------------+
|          term|    docid|             tfidf|
+--------------+---------+------------------+
|acceptableunto|bible-kjv| 1.349892008639309|
|adonijahthough|bible-kjv| 1.349892008639309|
|     ammonites|bible-kjv| 24.29805615550756|
|      andadmah|bible-kjv| 1.349892008639309|
|         anyof|bible-kjv| 4.724622030237581|
|      aphekand|bible-kjv| 1.349892008639309|
|     arguments|bible-kjv|0.2699784017278618|
|  arrayhimself|bible-kjv| 1.349892008639309|
|           art|bible-kjv| 75.25647948164146|
|      ashkenaz|bible-kjv| 1.349892008639309|
|        atroop|bible-kjv| 2.699784017278618|
|    avengedhim|bible-kjv| 1.349892008639309|
|      bedriven|bible-kjv| 1.349892008639309|
|   bemagnified|bible-kjv| 1.349892008639309|
|         besom|bible-kjv| 1.349892008639309|
|      blackish|bible-kjv|0.6749460043196545|
|       blossom|bible-kjv| 1.687365010799136|
|        brands|bible-kjv|0.6749460043196545|
|      burnwith|bible-kjv| 1.34989

In [46]:
from pprint import pprint
def relevence(str_to_check, tfidf_index):
    split_string = str_to_check.split()
    tokens = list(filter(lambda t: t != "", ["".join(filter(lambda c: 97 <= ord(c) <= 122, x.lower())) for x in split_string]))
    matched_tfidf = tfidf_index.filter(tfidf_index.term.isin(tokens)).groupBy("docid").sum("tfidf").withColumnRenamed('sum(tfidf)', "tfidf").sort(col("tfidf").desc()).take(5)
    
#     matched_tfidf = matched_tfidf.map(lambda t: (t[0], t[1]))
    
    return [(t[0], t[1]) for t in matched_tfidf]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
from pprint import pprint
for query in ["buster, whale, king, and alice the rabbit!",
               "Take a whale to lunch this week!",
              "What would Jesus do about that?",
              "My name is Buster.  Deal with it.",
              "Bodice ripper?",
              "What are leaves of grass anyway?",
              "??!?"]:
    print(query)
    pprint(relevence(query, myindex))
    print()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

buster, whale, king, and alice the rabbit!
[('carroll-alice', 20947.509101801927),
 ('burgess-busterbrown', 19291.14965380133),
 ('bible-kjv', 14077.483801295897),
 ('whitman-leaves', 12235.673736718632),
 ('blake-poems', 11262.520382017236)]

Take a whale to lunch this week!
[('melville-moby_dick', 6169.3860077853915),
 ('carroll-alice', 5557.859809207797),
 ('austen-emma', 5458.803580363596),
 ('chesterton-brown', 5247.206767087348),
 ('burgess-busterbrown', 4641.118804414889)]

What would Jesus do about that?
[('burgess-busterbrown', 2797.0110372249264),
 ('shakespeare-caesar', 2553.9314242243113),
 ('austen-emma', 2310.0016678712404),
 ('carroll-alice', 2260.47283284944),
 ('bible-kjv', 2229.2791576673867)]

My name is Buster.  Deal with it.
[('burgess-busterbrown', 15441.342095211978),
 ('shakespeare-caesar', 3958.2918244006114),
 ('blake-poems', 3437.4896010116136),
 ('whitman-leaves', 3334.75454989471),
 ('austen-emma', 3270.9690684894995)]

Bodice ripper?
[]

What are leaves of